In [1]:
# install joblib 
# Restart kernel after installing 
!pip install joblib

In [2]:
# !pip install tensorflow

In [3]:
# Update sklearn to prevent version mismatches
# !pip install sklearn --upgrade

In [4]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(1)

In [5]:
# Dependecies
%matplotlib inline
import matplotlib.pyplot as plt
import joblib
import numpy as np
import pandas as pd
import sklearn
import sklearn.datasets

In [6]:
# Read the billboard data into pandas
df2 = pd.read_csv('data/grouped_audio_billboard_genre_data.csv')

# Drop the null columns where all values are null
df2 = df2.dropna(axis='columns', how='all')

# Drop the null rows
df2 = df2.dropna()
df2.head()

# Display dataframe 
df2.head()

,track_id,genre,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,energy,key,loudness,mode,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,week_pos,instance,peak_pos,weeks_on_chart
0,"AdictoTainy, Anuel AA & Ozuna",pop,0.0,270740.0,91.0,0.734,0.836,10.0,-4.803,0.0,...,0.01700,0.000016,0.1790,0.623,80.002,4.0,98.0,1.0,86.0,2.0
1,The Ones That Didn't Make It Back HomeJustin M...,country,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.0,...,0.00000,0.000000,0.0000,0.000,0.000,0.0,100.0,1.0,100.0,1.0
2,ShallowLady Gaga & Bradley Cooper,pop,0.0,215733.0,89.0,0.572,0.385,7.0,-6.362,1.0,...,0.37100,0.000000,0.2310,0.323,95.799,4.0,28.0,1.0,28.0,1.0
3,EnemiesPost Malone Featuring DaBaby,rap,1.0,196760.0,86.0,0.542,0.674,6.0,-4.169,1.0,...,0.05880,0.000000,0.0955,0.667,76.388,4.0,98.0,1.0,16.0,15.0
4,"Bacc At It AgainYella Beezy, Gucci Mane & Quavo",rap,1.0,228185.0,61.0,0.948,0.623,8.0,-5.725,0.0,...,0.00124,0.000001,0.0716,0.856,135.979,4.0,94.0,1.0,94.0,3.0


In [7]:
# Assigning numbers to genre
df2['genre'] = df2['genre'].replace('pop',float(0)).replace('country',float(1))\
.replace('hiphop',float(2)).replace('other',float(3)).replace('latin',float(3)).replace('latin',float(4))\
.replace('house',float(5)).replace('folk',float(6)).replace('r&b',float(7)).replace('adult standards',float(8))\
.replace('rock',float(9)).replace('metal',float(10)).replace('show tunes',float(11)).replace('soul',float(12))\
.replace('rap',float(13)).replace('jazz',float(14))

In [38]:
# Discretize variable into equal-sized buckets
df2['target'] = pd.qcut(df2['peak_pos'],10, labels = False)
df2

,track_id,genre,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,energy,key,loudness,mode,...,instrumentalness,liveness,valence,tempo,time_signature,week_pos,instance,peak_pos,weeks_on_chart,target
0,"AdictoTainy, Anuel AA & Ozuna",0.0,0.0,270740.0,91.0,0.734,0.836,10.0,-4.803,0.0,...,0.000016,0.1790,0.623,80.002,4.0,98.0,1.0,86.0,2.0,4
1,The Ones That Didn't Make It Back HomeJustin M...,1.0,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.0,...,0.000000,0.0000,0.000,0.000,0.0,100.0,1.0,100.0,1.0,9
2,ShallowLady Gaga & Bradley Cooper,0.0,0.0,215733.0,89.0,0.572,0.385,7.0,-6.362,1.0,...,0.000000,0.2310,0.323,95.799,4.0,28.0,1.0,28.0,1.0,0
3,EnemiesPost Malone Featuring DaBaby,13.0,1.0,196760.0,86.0,0.542,0.674,6.0,-4.169,1.0,...,0.000000,0.0955,0.667,76.388,4.0,98.0,1.0,16.0,15.0,0
4,"Bacc At It AgainYella Beezy, Gucci Mane & Quavo",13.0,1.0,228185.0,61.0,0.948,0.623,8.0,-5.725,0.0,...,0.000001,0.0716,0.856,135.979,4.0,94.0,1.0,94.0,3.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28370,"Tonight, TonightThe Smashing Pumpkins",9.0,0.0,254626.0,69.0,0.287,0.670,6.0,-9.381,1.0,...,0.748000,0.2370,0.222,148.928,4.0,51.0,1.0,51.0,1.0,0
28371,Tonight's The Night (Gonna Be Alright)Rod Stewart,9.0,0.0,237120.0,55.0,0.550,0.514,11.0,-12.617,1.0,...,0.125000,0.3820,0.724,136.542,3.0,81.0,1.0,81.0,1.0,3
28372,Tonight's The NightBLACKstreet,0.0,0.0,258026.0,32.0,0.585,0.573,6.0,-5.561,0.0,...,0.000011,0.1730,0.591,76.060,4.0,95.0,2.0,95.0,2.0,7
28373,Tonight's The NightS.S.O.,3.0,0.0,170506.0,13.0,0.000,0.000,0.0,0.000,0.0,...,0.000000,0.0000,0.000,0.000,0.0,100.0,1.0,100.0,2.0,9


## Create a Train Test Split

Use `target`for the y values

In [9]:
from sklearn.model_selection import train_test_split
y = pd.Series(df2["target"])
X = df2.drop(columns=["track_id", "target" ])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [10]:
y.describe()

count    28375.000000
mean         4.339912
std          2.802909
min          0.000000
25%          2.000000
50%          4.000000
75%          7.000000
max          9.000000
Name: target, dtype: float64

In [11]:
X_train

,genre,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,week_pos,instance,peak_pos,weeks_on_chart
11734,9.0,0.0,245640.0,77.0,0.817,0.599,0.0,-9.249,0.0,0.0328,0.1320,0.000311,0.0873,0.548,108.873,4.0,73.0,1.0,73.0,1.0
28040,9.0,0.0,258399.0,63.0,0.569,0.744,8.0,-6.480,1.0,0.0298,0.0559,0.000003,0.0915,0.655,96.596,4.0,34.0,1.0,34.0,1.0
12045,3.0,0.0,240866.0,61.0,0.634,0.677,5.0,-7.278,0.0,0.0304,0.0117,0.001030,0.1260,0.558,135.012,4.0,67.0,1.0,67.0,1.0
14139,9.0,0.0,132280.0,0.0,0.714,0.464,8.0,-11.080,1.0,0.0514,0.9000,0.283000,0.1030,0.921,124.553,4.0,80.0,1.0,77.0,2.0
23223,8.0,0.0,135506.0,53.0,0.570,0.259,7.0,-14.393,1.0,0.0332,0.8050,0.000000,0.1120,0.485,124.863,4.0,100.0,1.0,100.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10955,14.0,0.0,189946.0,38.0,0.706,0.467,7.0,-9.523,1.0,0.0852,0.7130,0.000000,0.2530,0.735,107.053,4.0,63.0,1.0,63.0,1.0
17289,14.0,0.0,151866.0,7.0,0.652,0.656,7.0,-7.156,1.0,0.0967,0.2590,0.050900,0.1420,0.771,161.895,4.0,92.0,1.0,92.0,1.0
5192,9.0,0.0,162986.0,19.0,0.587,0.649,9.0,-9.411,1.0,0.0321,0.6050,0.000006,0.1390,0.831,96.111,4.0,89.0,1.0,89.0,1.0
12172,9.0,0.0,233506.0,63.0,0.626,0.565,9.0,-10.027,1.0,0.0271,0.1240,0.000000,0.0811,0.924,116.610,4.0,65.0,1.0,65.0,1.0


In [12]:
y_train

11734    2
28040    0
12045    1
14139    2
23223    9
        ..
10955    1
17289    6
5192     5
12172    1
235      0
Name: target, Length: 21281, dtype: int64

## Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [13]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Create a Deep Learning Model

In [14]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense

In [15]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [16]:
y_train_categorical

array([[0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [17]:
X_train_scaled.shape

(21281, 20)

In [18]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=10, activation='relu', input_dim=20))
model.add(Dense(units=10, activation='softmax'))

In [19]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                210       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
Total params: 320
Trainable params: 320
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
666/666 - 1s - loss: 0.5004 - accuracy: 0.9094
Epoch 2/100
666/666 - 1s - loss: 0.4978 - accuracy: 0.9120
Epoch 3/100
666/666 - 1s - loss: 0.4950 - accuracy: 0.9103
Epoch 4/100
666/666 - 1s - loss: 0.4935 - accuracy: 0.9144
Epoch 5/100
666/666 - 1s - loss: 0.4919 - accuracy: 0.9171
Epoch 6/100
666/666 - 1s - loss: 0.4904 - accuracy: 0.9160
Epoch 7/100
666/666 - 1s - loss: 0.4888 - accuracy: 0.9165
Epoch 8/100
666/666 - 1s - loss: 0.4870 - accuracy: 0.9210
Epoch 9/100
666/666 - 1s - loss: 0.4859 - accuracy: 0.9178
Epoch 10/100
666/666 - 1s - loss: 0.4842 - accuracy: 0.9166
Epoch 11/100
666/666 - 1s - loss: 0.4828 - accuracy: 0.9164
Epoch 12/100
666/666 - 1s - loss: 0.4815 - accuracy: 0.9190
Epoch 13/100
666/666 - 1s - loss: 0.4797 - accuracy: 0.9182
Epoch 14/100
666/666 - 1s - loss: 0.4787 - accuracy: 0.9204
Epoch 15/100
666/666 - 1s - loss: 0.4773 - accuracy: 0.9230
Epoch 16/100
666/666 - 1s - loss: 0.4761 - accuracy: 0.9186
Epoch 17/100
666/666 - 1s - loss: 0.4751 - accura

## Quantify our Trained Model

In [22]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

222/222 - 0s - loss: 0.3828 - accuracy: 0.9408
Normal Neural Network - Loss: 0.38284772634506226, Accuracy: 0.9407950639724731


## Save the Model

In [43]:
X_train.head()
# y_train.head()

,genre,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,week_pos,instance,peak_pos,weeks_on_chart
11734,9.0,0.0,245640.0,77.0,0.817,0.599,0.0,-9.249,0.0,0.0328,0.1320,0.000311,0.0873,0.548,108.873,4.0,73.0,1.0,73.0,1.0
28040,9.0,0.0,258399.0,63.0,0.569,0.744,8.0,-6.480,1.0,0.0298,0.0559,0.000003,0.0915,0.655,96.596,4.0,34.0,1.0,34.0,1.0
12045,3.0,0.0,240866.0,61.0,0.634,0.677,5.0,-7.278,0.0,0.0304,0.0117,0.001030,0.1260,0.558,135.012,4.0,67.0,1.0,67.0,1.0
14139,9.0,0.0,132280.0,0.0,0.714,0.464,8.0,-11.080,1.0,0.0514,0.9000,0.283000,0.1030,0.921,124.553,4.0,80.0,1.0,77.0,2.0
23223,8.0,0.0,135506.0,53.0,0.570,0.259,7.0,-14.393,1.0,0.0332,0.8050,0.000000,0.1120,0.485,124.863,4.0,100.0,1.0,100.0,1.0


In [23]:
model.save('Models/billb_spot_DL.h5')

In [53]:
minput = pd.DataFrame(
{'genre': 9,
'spotify_track_explicit': 0,
'spotify_track_duration_ms': 245640.0,
'spotify_track_popularity': 77,
'danceability': 0.817,
'energy': 0.599,
'key': 0,
'loudness': -9.249,
'mode': 0.0,
'speechiness': 0.0328,
'acousticness': 0.1320,
'instrumentalness': 0.000311,
'liveness': 0.0873,
'valence': 0.548,
'tempo': 108.873,
'time_signature': 4.0,
'week_pos': 73.0,
'instance': 1.0,
'peak_pos': 73.0,
'weeks_on_chart':1}, index = [0])
minput_scaled = X_scaler.transform(minput)
model.predict(minput_scaled,batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10,
    workers=1, use_multiprocessing=False)

array([[3.5518871e-03, 4.6140468e-01, 5.3023702e-01, 4.8064548e-03,
        3.9823197e-24, 1.6296268e-33, 8.6700341e-30, 4.0229640e-32,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [70]:
um = model(minput_scaled, training=False).numpy()

In [78]:
maxi = np.argmax(um)


2

In [57]:
from pickle import dump
dump(X_scaler, open('scaler.pkl', 'wb'))